In [1]:
using Enzyme
using ArrheniusModel

┌ Warning: attempting to remove probably stale pidfile
│   path = "C:\\Users\\heyye\\.julia\\compiled\\v1.10\\OrdinaryDiffEq\\DlSvy_VNblX.ji.pidfile"
└ @ FileWatching.Pidfile C:\Users\heyye\.julia\juliaup\julia-1.10.1+0.x64.w64.mingw32\share\julia\stdlib\v1.10\FileWatching\src\pidfile.jl:244
┌ Warning: attempting to remove probably stale pidfile
│   path = "C:\\Users\\heyye\\.julia\\compiled\\v1.10\\OrdinaryDiffEqDefault\\lMvuj_VNblX.ji.pidfile"
└ @ FileWatching.Pidfile C:\Users\heyye\.julia\juliaup\julia-1.10.1+0.x64.w64.mingw32\share\julia\stdlib\v1.10\FileWatching\src\pidfile.jl:244

Please submit a bug report with steps to reproduce this fault, and any error messages that follow (in their entirety). Thanks.
Exception: EXCEPTION_ACCESS_VIOLATION at 0x7fff5c298610 -- gc_setmark_pool_ at C:/workdir/src\gc.c:876 [inlined]
gc_setmark_pool at C:/workdir/src\gc.c:895 [inlined]
gc_setmark at C:/workdir/src\gc.c:902 [inlined]
gc_setmark at C:/workdir/src\gc.c:898 [inlined]
gc_mark_outrefs a

In [2]:

# example from tests
G = [1.0,0.0]
Ea = [0. 0.2; 0.2 0.]
pe = PhaseEnergies(G, Ea)

PhaseEnergies([1.0, 0.0], [0.0 0.2; 0.2 0.0], [0.0 0.2; 1.2 0.0])

In [7]:
# try forward mode without the struct first
# this will give the derivative of each element of the output matrix wrt the 1,2 element of the input
db = Array(zero(pe.barriers))
db[1,2] = 1.0
g12 = Enzyme.autodiff(Forward, arrhenius_rate, Duplicated(pe.barriers, db))[1]
# (convince yourself that this gives the right result)

2×2 Matrix{Float64}:
  0.0168865  -0.0168865
 -0.0         0.0

In [8]:

# similarly, to get the derivative wrt the 2,2 element we can do...
db = Array(zero(pe.barriers))
db[2,2] = 1.0
Enzyme.autodiff(Forward, arrhenius_rate, Duplicated(pe.barriers, db))
# (again, convince yourself)

(var"1" = [0.0 -0.0; -0.0 0.0],)

In [3]:
# we can do it wrt the temperature also...
db = Array(zero(pe.barriers))
dT = 1.0
Enzyme.autodiff(Forward, arrhenius_rate, Duplicated(pe.barriers, db), Duplicated(300.0, dT))
# (check this on paper too)

(var"1" = [-1.1257691092137462e-5 1.1257691092137462e-5; 1.0707553411056894e-21 -1.0707553411056894e-21],)

In [1]:
using OrdinaryDiffEq, Enzyme



┌ Warning: attempting to remove probably stale pidfile
│   path = C:\Users\heyye\.julia\compiled\v1.10\OrdinaryDiffEq\DlSvy_VNblX.ji.pidfile
└ @ FileWatching.Pidfile C:\Users\heyye\.julia\juliaup\julia-1.10.1+0.x64.w64.mingw32\share\julia\stdlib\v1.10\FileWatching\src\pidfile.jl:244


In [15]:
function fiip(du, u, p, t)
    du[1] = dx = p[1] * u[1] - p[2] * u[1] * u[2]
    du[2] = dy = -p[3] * u[2] + p[4] * u[1] * u[2]
end
p = [1.5, 1.0, 3.0, 1.0];
u0 = [1.0; 1.0];
prob = ODEProblem(fiip, u0, (0.0, 10.0), p)
sol = solve(prob, Tsit5())
loss(u0, p, prob) = sum(solve(prob, Tsit5(), u0 = u0, p = p, saveat = 0.1))
du = zeros(size(u0))
dp = zeros(size(p))
dp[1] = 1.0
println(loss(u0, p, prob))
gloss = Enzyme.autodiff(Reverse, loss, Active, Duplicated(u0, du), Duplicated(p, dp), Const(prob))
println(du)

450.6437728782925
[-39.49430913141475, -8.63188820376318]


In [17]:
du = zeros(size(u0))
dp = zeros(size(p))
dp[1] = 1.0
gloss = Enzyme.autodiff(Forward, loss, Duplicated(u0, du), Duplicated(p, dp), Const(prob))
gloss[1]


Enzyme.Compiler.EnzymeRuntimeActivityError: Constant memory is stored (or returned) to a differentiable variable.
As a result, Enzyme cannot provably ensure correctness and throws this error.
This might be due to the use of a constant variable as temporary storage for active memory (https://enzyme.mit.edu/julia/stable/faq/#Activity-of-temporary-storage).
If Enzyme should be able to prove this use non-differentable, open an issue!
To work around this issue, either:
 a) rewrite this variable to not be conditionally active (fastest, but requires a code change), or
 b) set Enzyme.API.runtimeActivity!(true) immediately after loading Enzyme (which maintains correctness, but may slightly reduce performance).
Mismatched activity for:   %value_phi = select i1 %.not, {} addrspace(10)* %getfield64, {} addrspace(10)* %getfield, !dbg !205 const val:   %getfield64 = load atomic {} addrspace(10)*, {} addrspace(10)* addrspace(11)* %9 unordered, align 8, !dbg !205
Type tree: {[-1]:Pointer, [-1,0]:Point

In [16]:
dp

4-element Vector{Float64}:
    8.349039360098285
 -159.3107987785755
   74.93924800431942
 -339.3272380777995